In [1]:
import numpy as np
import cv2 as cv
import tiffile as tiff
import random
import anndata as ad
from skimage.measure import shannon_entropy
from skimage.transform import integral_image
from skimage.feature import graycomatrix, graycoprops

random.seed(0)
np.random.seed(0)

In [2]:
# load the anndata file with the SIFT descriptors
adata_filename = '/gladstone/engelhardt/lab/adamw/saft_figuren/analysis/adata_20250225_kmeans.h5ad'
adata = ad.read_h5ad(adata_filename)

# reset the index of the obs dataframe
adata.obs = adata.obs.reset_index(drop=True)

/gladstone/engelhardt/home/adamw/miniforge3/envs/saft/lib/python3.11/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


### Create functions for computing the RFP intensity statistics surrounding each ROI

In [3]:


def compute_roi_boundaries(rfp_image, x, y, scales, octaves):
    """
    Compute ROI boundaries for arrays of x, y, scales, and octaves.
    
    The ROI is a square centered at (x,y) with half-size given by
      radius = scales * (2 ** (octaves + 1)).
    """
    radii = scales * (2 ** (octaves + 1))
    xmin = np.clip(np.floor(x - radii).astype(int), 0, rfp_image.shape[0])
    xmax = np.clip(np.ceil(x + radii).astype(int), 0, rfp_image.shape[0])
    ymin = np.clip(np.floor(y - radii).astype(int), 0, rfp_image.shape[1])
    ymax = np.clip(np.ceil(y + radii).astype(int), 0, rfp_image.shape[1])
    return xmin, xmax, ymin, ymax

def compute_mean_intensities(rfp_image, df):
    """
    Compute mean intensity for each ROI in the DataFrame using an integral image.
    
    Parameters:
      rfp_image : 2D numpy array for the RFP channel.
      df        : DataFrame containing columns 'x', 'y', 'scales', and 'octaves'.
      
    Returns:
      A list of mean intensity values (one per ROI).
    """
    # Extract ROI parameters as arrays.
    x = df['x'].values
    y = df['y'].values
    scales = df['scales'].values
    octaves = df['octaves'].values
    
    xmin, xmax, ymin, ymax = compute_roi_boundaries(rfp_image, x, y, scales, octaves)
    
    # Compute the integral image once.
    ii = integral_image(rfp_image)
    
    mean_intensity = []
    for i in range(len(x)):
        x_min = xmin[i]
        x_max = xmax[i]
        y_min = ymin[i]
        y_max = ymax[i]
        area = (x_max - x_min) * (y_max - y_min)
        if area <= 0:
            mean_intensity.append(0)
        else:
            # Compute the sum using four look-ups.
            A = ii[x_max-1, y_max-1] if (x_max-1 >= 0 and y_max-1 >= 0) else 0
            B = ii[x_min-1, y_max-1] if x_min-1 >= 0 else 0
            C = ii[x_max-1, y_min-1] if y_min-1 >= 0 else 0
            D = ii[x_min-1, y_min-1] if (x_min-1 >= 0 and y_min-1 >= 0) else 0
            sum_intensity = A - B - C + D
            mean_intensity.append(sum_intensity / area)
    return mean_intensity

def compute_entropy_for_roi(rfp_image, row):
    """
    Compute the Shannon entropy for a single ROI defined in the row.
    """
    radius = row['scales'] * (2 ** (row['octaves'] + 1))
    x_min = int(np.clip(np.floor(row['x'] - radius), 0, rfp_image.shape[0]))
    x_max = int(np.clip(np.ceil(row['x'] + radius), 0, rfp_image.shape[0]))
    y_min = int(np.clip(np.floor(row['y'] - radius), 0, rfp_image.shape[1]))
    y_max = int(np.clip(np.ceil(row['y'] + radius), 0, rfp_image.shape[1]))
    roi = rfp_image[x_min:x_max, y_min:y_max]
    return shannon_entropy(roi)

def compute_entropies(rfp_image, df):
    """
    Compute Shannon entropies for all ROIs in the DataFrame.
    
    Uses a DataFrame.apply call to process each ROI.
    """
    return df.apply(lambda row: compute_entropy_for_roi(rfp_image, row), axis=1)


### Create functions for computing the grey level correlation matrix (GLCM) statistics surrounding each ROI

In [4]:
def compute_glcm_for_roi(image, row):
    """
    Compute the GLCM for a single ROI defined in the row.
    """
    radius = row['scales'] * (2 ** (row['octaves'] + 1))
    x_min = int(np.clip(np.floor(row['x'] - radius), 0, image.shape[0]))
    x_max = int(np.clip(np.ceil(row['x'] + radius), 0, image.shape[0]))
    y_min = int(np.clip(np.floor(row['y'] - radius), 0, image.shape[1]))
    y_max = int(np.clip(np.ceil(row['y'] + radius), 0, image.shape[1]))
    roi = image[x_min:x_max, y_min:y_max]

    texture_mat = graycomatrix(roi,
                               distances=[5],
                               angles=[0],
                               levels=256,
                               symmetric=True,
                               normed=True)
    glcm_homogeneity = graycoprops(texture_mat, 'homogeneity')[0, 0]
    glcm_energy = graycoprops(texture_mat, 'energy')[0, 0]

    return glcm_homogeneity, glcm_energy

def compute_glcm(image, df):
    """
    Compute the GLCM for all ROIs in the DataFrame.
    
    Uses a DataFrame.apply call to process each ROI.
    """
    results = df.apply(lambda row: compute_glcm_for_roi(image, row), axis=1)
    glcm_homogeneities, glcm_energies = zip(*results)
    return list(glcm_homogeneities), list(glcm_energies)

### Loop through all the ROIs in the adata object and compute their RFP and GLCM statistics

Group rows by filename when looping over ROIs to ensure that we load each image file only once.

In [5]:
def load_image(row, rfp=False):
    """
    Load an image based on the filename provided in the row.
    If rfp is True, load the corresponding RFP channel image.
    """
    if rfp:
        filename = row['filename'].replace('phase_registered', 'red_registered')
        image = tiff.imread(filename)
    else:
        filename = row['filename']
        image = tiff.imread(filename)
        # Only normalize the brightfield image to the range [0, 255]
        image = cv.normalize(image, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
    return image

# ========================================================
# Main loop: Process each image file (grouped by filename)
# ========================================================
for bf_path, image_df in adata.obs.groupby('filename'):

    # Load the brightfield (BF) and RFP images only once for this group.
    bf_image = load_image(image_df.iloc[0])
    rfp_image = load_image(image_df.iloc[0], rfp=True)
    
    # Compute mean RFPintensities for all ROIs in this image.
    mean_intensities = compute_mean_intensities(rfp_image, image_df)
    
    # Compute RFP entropies for all ROIs in this image.
    entropies = compute_entropies(rfp_image, image_df)

    # Compute GLCM stats for all ROIs in this image.
    glcm_homogeneity, glcm_energy = compute_glcm(bf_image, image_df)
    
    # Update the main DataFrame using .loc with the image_df indices.
    adata.obs.loc[image_df.index, 'roi_mean_rfp_intensity'] = mean_intensities
    adata.obs.loc[image_df.index, 'roi_rfp_entropy'] = entropies
    adata.obs.loc[image_df.index, 'roi_glcm_homogeneity'] = glcm_homogeneity
    adata.obs.loc[image_df.index, 'roi_glcm_energy'] = glcm_energy

adata.obs.head()

/tmp/ipykernel_2646343/628602981.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for bf_path, image_df in adata.obs.groupby('filename'):


,donor_id,time,well_id,rasa2ko_titration,et_ratio,entropy,p_areas,filename,scales,octaves,...,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10,roi_mean_rfp_intensity,roi_rfp_entropy,roi_glcm_homogeneity,roi_glcm_energy
0,1,0,B2,100.0,2.8284,4.017321,27030,/gladstone/engelhardt/lab/MarsonLabIncucyteDat...,2,0,...,0,4,5,6,3,8,0.806531,4.324994,0.063431,0.147314
1,1,0,B2,100.0,2.8284,4.017321,27030,/gladstone/engelhardt/lab/MarsonLabIncucyteDat...,1,0,...,4,3,3,5,4,2,0.273000,2.655639,0.000000,0.000000
2,1,0,B2,100.0,2.8284,4.017321,27030,/gladstone/engelhardt/lab/MarsonLabIncucyteDat...,1,0,...,0,4,0,2,1,3,5.436375,4.000000,0.000000,0.000000
3,1,0,B2,100.0,2.8284,4.017321,27030,/gladstone/engelhardt/lab/MarsonLabIncucyteDat...,2,0,...,3,0,2,1,2,6,1.303312,5.020765,0.000548,0.144338
4,1,0,B2,100.0,2.8284,4.017321,27030,/gladstone/engelhardt/lab/MarsonLabIncucyteDat...,1,0,...,2,2,4,3,0,5,3.379687,3.750000,0.000000,0.000000


In [6]:
# save the adata object
output_filename = adata_filename.replace('_kmeans.h5ad', '_processed.h5ad')
adata.write(output_filename)